In [14]:
#############

import numpy as np

# Fixed random seed for reproducibility
np.random.seed(12345)

# Number of samples
N = 3_000_000

# Means, stdevs
mu_x_0, mu_y_0 = 1.2, 1.1
sigma_x_0, sigma_y_0 = 1.0, 1.0

# Proposed solution
x_star_0 = 0.68
y_star_0 = 0.78
c_0 = 0.9

# Slope of line D that passes through (x_star, y_star) and (2,2)
#   slope = (2 - y_star)/(2 - x_star)
slope_D_0 = (c_0 - y_star_0) / (c_0 - x_star_0)

def yD_of_x(x):
    return y_star_0 + slope_D_0 * (x - x_star_0)
def xD_of_y(y):
    # slope = dy/dx => dx/dy = 1/slope
    return x_star_0 + (1./slope_D_0) * (y - y_star_0)

# 1) Generate random samples from the uniform distributions
X_0 = np.random.uniform(0,1.1,N)
Y_0 = np.random.uniform(0,1,N)

# 2) Define region B: 
#    "above line F" => y >= y_star if x <= x_star,
#    "above line D" => y >= yD_of_x(x) if x >= x_star
#    Combined logically:
left_mask_B_0  = (X_0 <= x_star_0) & (Y_0 >= y_star_0)
right_mask_B_0 = (X_0 >= x_star_0) & (Y_0 >= yD_of_x(X_0))
B_mask_0 = left_mask_B_0 | right_mask_B_0

# 3) Define region A:
#    "right of line E" => x >= x_star if y <= y_star,
#    "right of line D" => x >= xD_of_y(y) if y >= y_star
#    Combined logically:
down_mask_A_0  = (Y_0 <= y_star_0) & (X_0 >= x_star_0)
up_mask_A_0    = (Y_0 >= y_star_0) & (X_0 >= xD_of_y(Y_0))
A_mask_0 = down_mask_A_0 | up_mask_A_0

# 4) Estimate E[Y | B]
Y_in_B_0 = Y_0[B_mask_0]
meanY_in_B_0 = np.mean(Y_in_B_0) if len(Y_in_B_0) > 0 else float('nan')

# 5) Estimate E[X | A]
X_in_A_0 = X_0[A_mask_0]
meanX_in_A_0 = np.mean(X_in_A_0) if len(X_in_A_0) > 0 else float('nan')


print("INDEPENDENT UNIFORMS")
print("Share of X in A =", len(X_in_A_0)/N)
print("Share of Y in B =", len(Y_in_B_0)/N)
print("x* =", x_star_0)
print("y* =", y_star_0)
print("E(x) in A =", meanX_in_A_0)
print("E(y) in B =", meanY_in_B_0)



INDEPENDENT UNIFORMS
Share of X in A = 0.3417226666666667
Share of Y in B = 0.17646733333333334
x* = 0.68
y* = 0.78
E(x) in A = 0.8988003823768471
E(y) in B = 0.8985234987586532


In [15]:
############################## joining c;c ##############################

# Initialize x_star and y_star
x_star_1 = 0
y_star_1 = 0

# Define the target mean value
target_mean_1 = 0.9

# Define a tolerance for convergence
tolerance_1 = 1e-7

# Define a maximum number of iterations to prevent infinite loops
max_iterations_1 = 1000

# Function to update x_star and y_star
def update_stars(x_star, y_star, meanX_in_A, meanY_in_B, learning_rate=0.02):
    x_star += learning_rate * (target_mean_1 - meanX_in_A)
    y_star += learning_rate * (target_mean_1 - meanY_in_B)
    return x_star, y_star

# Iteratively update x_star and y_star until convergence
for iteration_1 in range(max_iterations_1):
    # Update slope_D based on current x_star and y_star
    slope_D_1 = (target_mean_1 - y_star_1) / (target_mean_1 - x_star_1)
    
    def yD_of_x(x):
        return y_star_1 + slope_D_1 * (x - x_star_1)
    
    def xD_of_y(y):
        return x_star_1 + (1./slope_D_1) * (y - y_star_1)
    
    # Define region B
    left_mask_B_1  = (X_0 <= x_star_1) & (Y_0 >= y_star_1)
    right_mask_B_1 = (X_0 >= x_star_1) & (Y_0 >= yD_of_x(X_0))
    B_mask_1 = left_mask_B_1 | right_mask_B_1
    
    # Define region A
    down_mask_A_1  = (Y_0 <= y_star_1) & (X_0 >= x_star_1)
    up_mask_A_1    = (Y_0 >= y_star_1) & (X_0 >= xD_of_y(Y_0))
    A_mask_1 = down_mask_A_1 | up_mask_A_1
    
    # Estimate E[Y | B]
    Y_in_B_1 = Y_0[B_mask_1]
    meanY_in_B_1 = np.mean(Y_in_B_1) if len(Y_in_B_1) > 0 else float('nan')
    
    # Estimate E[X | A]
    X_in_A_1 = X_0[A_mask_1]
    meanX_in_A_1 = np.mean(X_in_A_1) if len(X_in_A_1) > 0 else float('nan')
    
    # Check for convergence
    if abs(meanY_in_B_1 - target_mean_1) < tolerance_1 and abs(meanX_in_A_1 - target_mean_1) < tolerance_1:
        break
    
    # Update x_star and y_star
    x_star_1, y_star_1 = update_stars(x_star_1, y_star_1, meanX_in_A_1, meanY_in_B_1)

print("ITERATIONS ON UNIFORMS")
print("")
print("Converged after", iteration_1 + 1, "iterations")
print("Share of X in A =", len(X_in_A_1)/N)
print("Share of Y in B =", len(Y_in_B_1)/N)
print("Total area covered=", (len(X_in_A_1) + len(Y_in_B_1))/N)
print("x* =", x_star_1)
print("y* =", y_star_1)
print("E(x) in A =", meanX_in_A_1)
print("E(y) in B =", meanY_in_B_1)

ITERATIONS ON UNIFORMS

Converged after 1000 iterations
Share of X in A = 0.339865
Share of Y in B = 0.17446733333333334
Total area covered 0.5143323333333333
x* = 0.682671794976199
y* = 0.7831154600373038
E(x) in A = 0.8999231843576166
E(y) in B = 0.8999336858105957


In [16]:
######################## joining 1;1 ########################

# Initialize x_star and y_star
x_star_2 = 0
y_star_2 = 0

# Define the target mean value
target_mean_2 = 0.9

# Define a tolerance for convergence
tolerance_2 = 1e-7

# Define a maximum number of iterations to prevent infinite loops
max_iterations_2 = 1000

# Function to update x_star and y_star
def update_stars(x_star, y_star, meanX_in_A, meanY_in_B, learning_rate=0.02):
    x_star += learning_rate * (target_mean_2 - meanX_in_A)
    y_star += learning_rate * (target_mean_2 - meanY_in_B)
    return x_star, y_star

# Iteratively update x_star and y_star until convergence
for iteration_2 in range(max_iterations_2):
    # Update slope_D based on current x_star and y_star
    slope_D_2 = (1 - y_star_2) / (1 - x_star_2)
    
    def yD_of_x(x):
        return y_star_2 + slope_D_2 * (x - x_star_2)
    
    def xD_of_y(y):
        return x_star_2 + (1./slope_D_2) * (y - y_star_2)
    
    # Define region B
    left_mask_B_2  = (X_0 <= x_star_2) & (Y_0 >= y_star_2)
    right_mask_B_2 = (X_0 >= x_star_2) & (Y_0 >= yD_of_x(X_0))
    B_mask_2 = left_mask_B_2 | right_mask_B_2
    
    # Define region A
    down_mask_A_2  = (Y_0 <= y_star_2) & (X_0 >= x_star_2)
    up_mask_A_2    = (Y_0 >= y_star_2) & (X_0 >= xD_of_y(Y_0))
    A_mask_2 = down_mask_A_2 | up_mask_A_2
    
    # Estimate E[Y | B]
    Y_in_B_2 = Y_0[B_mask_2]
    meanY_in_B_2 = np.mean(Y_in_B_2) if len(Y_in_B_2) > 0 else float('nan')
    
    # Estimate E[X | A]
    X_in_A_2 = X_0[A_mask_2]
    meanX_in_A_2 = np.mean(X_in_A_2) if len(X_in_A_2) > 0 else float('nan')
    
    # Check for convergence
    if abs(meanY_in_B_2 - target_mean_2) < tolerance_2 and abs(meanX_in_A_2 - target_mean_2) < tolerance_2:
        break
    
    # Update x_star and y_star
    x_star_2, y_star_2 = update_stars(x_star_2, y_star_2, meanX_in_A_2, meanY_in_B_2)

print("ITERATIONS ON UNIFORMS")
print("")
print("Converged after", iteration_2 + 1, "iterations")
print("Share of X in A =", len(X_in_A_2)/N)
print("Share of Y in B =", len(Y_in_B_2)/N)
print("Total area covered=", len(X_in_A_2)/N + len(Y_in_B_2)/N)
print("x* =", x_star_2)
print("y* =", y_star_2)
print("E(x) in A =", meanX_in_A_2)
print("E(y) in B =", meanY_in_B_2)

ITERATIONS ON UNIFORMS

Converged after 1000 iterations
Share of X in A = 0.3494773333333333
Share of Y in B = 0.163635
Total area covered= 0.5131123333333333
x* = 0.681822129983563
y* = 0.7861060824689039
E(x) in A = 0.899928831670676
E(y) in B = 0.8999291350958003


In [17]:
######################## joining max;max ########################

# Initialize x_star and y_star
x_star_3 = 0
y_star_3 = 0

# Define the target mean value
target_mean_3 = 0.9

# Define a tolerance for convergence
tolerance_3 = 1e-7

# Define a maximum number of iterations to prevent infinite loops
max_iterations_3 = 1000

# Function to update x_star and y_star
def update_stars(x_star, y_star, meanX_in_A, meanY_in_B, learning_rate=0.02):
    x_star += learning_rate * (target_mean_3 - meanX_in_A)
    y_star += learning_rate * (target_mean_3 - meanY_in_B)
    return x_star, y_star

# Iteratively update x_star and y_star until convergence
for iteration_3 in range(max_iterations_3):
    # Update slope_D based on current x_star and y_star
    slope_D_3 = (1 - y_star_3) / (1.1 - x_star_3)
    
    def yD_of_x(x):
        return y_star_3 + slope_D_3 * (x - x_star_3)
    
    def xD_of_y(y):
        return x_star_3 + (1./slope_D_3) * (y - y_star_3)
    
    # Define region B
    left_mask_B_3  = (X_0 <= x_star_3) & (Y_0 >= y_star_3)
    right_mask_B_3 = (X_0 >= x_star_3) & (Y_0 >= yD_of_x(X_0))
    B_mask_3 = left_mask_B_3 | right_mask_B_3
    
    # Define region A
    down_mask_A_3  = (Y_0 <= y_star_3) & (X_0 >= x_star_3)
    up_mask_A_3    = (Y_0 >= y_star_3) & (X_0 >= xD_of_y(Y_0))
    A_mask_3 = down_mask_A_3 | up_mask_A_3
    
    # Estimate E[Y | B]
    Y_in_B_3 = Y_0[B_mask_3]
    meanY_in_B_3 = np.mean(Y_in_B_3) if len(Y_in_B_3) > 0 else float('nan')
    
    # Estimate E[X | A]
    X_in_A_3 = X_0[A_mask_3]
    meanX_in_A_3 = np.mean(X_in_A_3) if len(X_in_A_3) > 0 else float('nan')
    
    # Check for convergence
    if abs(meanY_in_B_3 - target_mean_3) < tolerance_3 and abs(meanX_in_A_3 - target_mean_3) < tolerance_3:
        break
    
    # Update x_star and y_star
    x_star_3, y_star_3 = update_stars(x_star_3, y_star_3, meanX_in_A_3, meanY_in_B_3)

print("ITERATIONS ON UNIFORMS")
print("")
print("Converged after", iteration_3 + 1, "iterations")
print("Share of X in A =", len(X_in_A_3)/N)
print("Share of Y in B =", len(Y_in_B_3)/N)
print("Total area covered=", len(X_in_A_3)/N + len(Y_in_B_3)/N)
print("x* =", x_star_3)
print("y* =", y_star_3)
print("E(x) in A =", meanX_in_A_3)
print("E(y) in B =", meanY_in_B_3)

ITERATIONS ON UNIFORMS

Converged after 1000 iterations
Share of X in A = 0.338031
Share of Y in B = 0.17626566666666665
Total area covered= 0.5142966666666666
x* = 0.6831222246167095
y* = 0.7826559828989648
E(x) in A = 0.8999292254802179
E(y) in B = 0.8999290669203183
